## 1. Import libraries

In [1]:
import sys
sys.path.append('../')  # Enable importing from `src` folder

%matplotlib inline
from collections import defaultdict
from typing import List

import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns

import gym

import src.agents as agents
import src.episode as episode
import src.environment as environment
import src.aux_plots as aux_plots

## 2. Auxiliary functions

### 2.1. Reduce state space

In [2]:
def reduce_state(ob):
    # Doesn't matter where we were hit
    ob[16] = 1 if ob[16] != 255 else 0

    # Reduce chicken y-position
    ob[14] = ob[14] // 3

    for b in range(108, 118):
        # The chicken is in the x-posistion ~49
        if ob[b] < 20 or ob[b] > 80:
            # We don't need to represent cars far from the chicken
            ob[b] = 0
        else:
            # Reduce the cars x-positions sample space
            ob[b] = ob[b] // 3

    return ob

### 2.2. Reward policy

In [3]:
def reward_policy(reward, ob, action):
    if reward == 1:
        reward = reward_policy.REWARD_IF_CROSS
    elif ob[16] == 1:  # Collision!
        reward = reward_policy.REWARD_IF_COLISION
    elif action != 1:  # Don't incentivate staying still
        reward = reward_policy.REWARD_IF_STILL

    return reward

### 2.3. Print results

In [4]:
def print_result(i, scores, total_reward, score):
    if i % 10 == 0:
        print(f"Run [{i:4}] - Total reward: {total_reward:7.2f} Mean scores: {sum(scores) / len(scores):.2f} Means Scores[:-10]: {sum(scores[-10:]) / len(scores[-10:]):5.2f} Score: {score:2} ")

## 3. Create tiles

Tiles are a way of discretize features, locating them in grids.

Reference: [Tile Coding](https://towardsdatascience.com/reinforcement-learning-tile-coding-implementation-7974b600762b)

In [48]:
def create_tile(feature_range, n_bins=10, offset=0):
    return np.linspace(feature_range[0], feature_range[1], n_bins+1)[1:-1] + offset


def create_multiple_tiles(feature_ranges, n_tilings, n_bins, offsets):
    
    tilings = []
    # for each tiling
    for tile_i in range(n_tilings):
        tiling_bin = n_bins[tile_i]
        tiling_offset = offsets[tile_i]
        
        tiling = []
        # for each feature dimension
        for feat_i in range(len(feature_ranges)):
            feat_range = feature_ranges[feat_i]
            # tiling for 1 feature
            feat_tiling = create_tile(feat_range, tiling_bin[feat_i], tiling_offset[feat_i])
            tiling.append(feat_tiling)
        tilings.append(tiling)
    return np.array(tilings)

def get_tile_coding(feature, tilings):
    num_dims = len(feature)
    feat_codings = []
    for tiling in tilings:
        feat_coding = []
        for i in range(num_dims):
            feat_i = feature[i]
            tiling_i = tiling[i]  # tiling on that dimension
            coding_i = np.digitize(feat_i, tiling_i)
            feat_coding.append(coding_i)
        feat_codings.append(feat_coding)
    return np.array(feat_codings)

In [45]:
features_ranges = [[0,255], [0,1], [10,25]]
n_tiles = 4
n_bins = [[10 for f in range(len(features_ranges))] for t in range(n_tiles)]
rand = np.sort([0]+[np.random.randint(10) for t in range(n_tiles-1)])
offsets = [[t+r for f in range(len(features_ranges))] for t,r in zip(range(n_tiles), rand)]

In [46]:
tiles = create_multiple_tiles(features_ranges, n_tiles, n_bins, offsets)

In [55]:
features = [173, 0.25, 14]

In [56]:
codings = get_tile_coding(features, tiles)

In [57]:
codings

array([[6, 2, 2],
       [6, 0, 2],
       [6, 0, 0],
       [6, 0, 0]])

In [39]:
offsets

[[0, 0, 0], [1, 1, 1], [4, 4, 4], [11, 11, 11]]

In [58]:
rand_num = np.random.normal(0,1,15)

In [61]:
x = np.ones((len(rand_num)))

In [60]:
rand_num.shape

(15,)

In [62]:
np.dot(x, rand_num,)

3.9208481221775346

In [64]:
np.sum(rand_num)

3.9208481221775346

In [65]:
def getApproximation(W, feature):
        return np.dot(feature, W)

In [66]:
getApproximation(x, rand_num)

3.9208481221775346